 <span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 42px;">Covid-19 pandemic Analysis</span>

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 30px;">Database Description</span>

<span style="font-family: 'Times New Roman', Times, serif; font-size: 18px;"> Database consists of two tables: (**covid_analysis, population**)</span>


<span style="font-family: 'Times New Roman', Times, serif ; font-size: 20px;">covid_analysis:</span>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| Column | Description |
|----------|----------|
| date | A unique Identifier for each day |
| location | Every country / continent / four income groups|
| new_cases | new cases recorded per day |
| death_cases | new death cases recorded per day|
| total_cases | Cumulative cases recorded for a specifc country|
| total_deaths | Cumulative deaths recorded for a specifc country |
| weekly_cases | 7-days total cases|
| weekly_deaths | 7-days total death cases |
| biweekly_cases | 14-days total cases |
| biweekly_deaths | 14-days total death cases |


<span style="font-family: 'Times New Roman', Times, serif ; font-size: 20px;">population:</span>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| Column | Description |
|----------|----------|
| country_name | A unique Identifier for each country / region / world |
| country_code | short alphanumeric identification code for countries |
| year | year corresponds to each country and population |
| value | population corresponds to each country and year |

<br><span style="font-family: 'Times New Roman', Times, serif; font-size: 23px;">**NOTE** Database contains recorded data of covid-19 from (**22-01-2020 to 09-03-2023**)</span>
<br>
<br>
<br>
<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Tables links:  </span>
<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">[covid_analysis](https://github.com/omar25599/SQL-Projects/blob/main/Covid-19%20pandemic%20Analysis/full_data.csv)</span>,
<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">[population](https://github.com/omar25599/SQL-Projects/blob/main/Covid-19%20pandemic%20Analysis/population.csv)</span>

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Tables Creation & Importing Data</span>

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 23px;">1<sup>st</sup>  covid_analysis:</span>

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">Table Creation:</span>

In [ ]:
CREATE TABLE covid_analysis(
    date DATE NOT NULL,
    location VARCHAR(150),
    new_cases INTEGER,
    death_cases INTEGER,
    total_cases BIGINT,
    total_deaths BIGINT,
    weekly_cases INTEGER,
    weekly_deaths INTEGER,
    biweekly_cases INTEGER,
    biweekly_deaths INTEGER
);

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">Data Importing:</span>

In [ ]:
COPY covid_analysis FROM 'F:\markmarkoh-coronavirus-data\data\full_data.csv' DELIMITER ',' CSV HEADER;

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 23px;">2<sup>nd</sup>  population:</span>

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">Table Creation:</span>

In [ ]:
CREATE TABLE population(
    country_name VARCHAR(150),
    country_code VARCHAR(20),
    year INTEGER,
    value BIGINT
);

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">Data Importing:</span>

In [ ]:
COPY population FROM 'F:\covid_analysis\population.csv' DELIMITER ',' CSV HEADER;

<br>
<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Data Cleaning</span>

In [ ]:
--date is clean
SELECT COUNT(*)
FROM covid_analysis
WHERE date IS NULL;

--Replacing null values
--new_cases
UPDATE covid_analysis
SET new_cases = CASE WHEN new_cases IS NULL THEN 0 ELSE new_cases END;
--death_cases
UPDATE covid_analysis 
SET death_cases = CASE WHEN death_cases IS NULL THEN 0 ELSE death_cases END;

<br>
<br>
<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Objectives</span>
<br>
<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">1- In which year does the highest number of death cases occur ? In which Country Specify it as a percent of the population in the same year? What is the total number of death cases?</span>

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Solution</span>

In [ ]:
WITH req_loc_year AS(
    SELECT EXTRACT(YEAR FROM date) AS year, location
        FROM covid_analysis
        WHERE location IN (
            SELECT location
            FROM covid_analysis
            WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
            GROUP BY location
            ORDER BY SUM(death_cases) DESC
            LIMIT 1)
        GROUP BY EXTRACT(YEAR FROM date), location
        ORDER BY SUM(death_cases) DESC
        LIMIT 1
)
SELECT p.year AS most_death_year,
location AS country, 
(SELECT SUM(death_cases) AS no_of_death_cases
       FROM covid_analysis
       LEFT JOIN req_loc_year
       USING(location)
       WHERE EXTRACT(YEAR FROM date) = req_loc_year.year 
       AND location = req_loc_year.location),
p.value AS population,
ROUND((SELECT SUM(death_cases) AS no_of_death_cases
       FROM covid_analysis
       LEFT JOIN req_loc_year
       USING(location)
       WHERE EXTRACT(YEAR FROM date) = req_loc_year.year 
       AND location = req_loc_year.location) / CAST(p.value AS NUMERIC), 5) AS index,
ROUND((SELECT SUM(death_cases) AS no_of_death_cases
       FROM covid_analysis
       LEFT JOIN req_loc_year
       USING(location)
       WHERE EXTRACT(YEAR FROM date) = req_loc_year.year 
       AND location = req_loc_year.location) / CAST(p.value AS NUMERIC), 5) * 100 AS percentage,
SUM(death_cases) AS total_death_cases,
ROUND(SUM(death_cases) / CAST(p.value AS NUMERIC), 5) AS total_index,
ROUND(SUM(death_cases) / CAST(p.value AS NUMERIC), 5) * 100 AS total_percentage
FROM covid_analysis AS c
LEFT JOIN population AS p
ON c.location = p.country_name
WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
AND p.year IN 
       (
        SELECT year
        FROM req_loc_year
       )
GROUP BY location, p.value, p.year
ORDER BY total_death_cases DESC
LIMIT 1;

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Output</span>
<br>
<br>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| most_death_year | country | no_of_death_cases | population | index | percentage | total_death_cases | total_index | total_percentage |
|----------|----------|----------|----------|----------|----------|----------|----------|----------|
| 2021 | United States | 474864 | 331893745 | 0.00143 | 0.14300 | 1124389 | 0.00339 | 0.33900 |

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">insights from this query result :</span>
<br>
<span style="font-family: 'Times New Roman', Times, serif; font-size: 20px;">1- **united states** is the most country in death cases with **1124389** death cases represents **.339** percent of population in year **2021**</span>
<br>
<span style="font-family: 'Times New Roman', Times, serif; font-size: 20px;">2- **2021** is the most year in death cases in **USA** with **474864** death cases and this represents **.143** percent of population in the same year</span>

------------------------------------------------------------------------------------------------------------------------------

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">2- What is the total cases & death cases of covid-19 recorded in each country ranked by most spread? How much people survived & what is the representing percentage of them? </span>

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Solution</span>

In [ ]:
SELECT  ROW_NUMBER() OVER(ORDER BY SUM(new_cases) DESC) AS ranking, location, 
            SUM(new_cases) AS total_cases, SUM(death_cases) AS total_deaths,
            SUM(new_cases) - SUM(death_cases) AS count_of_cases_survived,
            ROUND(CAST(SUM(death_cases) AS NUMERIC) / SUM(new_cases), 5) * 100 AS percentage_of_death,
            (1 - ROUND(CAST(SUM(death_cases) AS NUMERIC) / SUM(new_cases), 5)) * 100 AS percentage_of_recovery
    FROM covid_analysis
    WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
    GROUP BY  location
    ORDER BY SUM(new_cases) DESC
    LIMIT 10;

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Output</span>
<br>
<br>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| ranking | location | total_cases | total_deaths | count_of_cases_survived | percentage_of_death | percentage_of_recovery |
|----------|----------|----------|----------|----------|----------|----------|
| 1 | United States | 103806563 | 1124389 | 102682174 | 1.08300 | 98.91700 |
| 2 | India | 44693901 | 523406 | 44170495 | 1.17100 | 98.82900 |
| 3 | France | 40155274 | 165613 | 39989661 |0.41200 | 99.58800 |
| 4 | Germany | 38249060 | 168944 | 38080116 | 0.44200 | 99.55800 |
| 5 | Brazil | 36977190 | 699402 | 36277788 | 1.89100 | 98.10900 |
| 6 | Japan | 33320436 | 73007 | 33247429 | 0.21900 | 99.78100 |
| 7 | South Korea | 30615521 | 34093 | 30581428 | 0.11100 | 99.88900 |
| 8 | Italy | 25603658 | 188353 | 25415305 | 0.73600 | 99.26400 |
| 9 | United Kingdom | 23589101 | 219341 | 23369760 | 0.93000 | 99.07000 |
| 10 | Russia | 22075858 | 388478 | 21687380 | 1.76000 | 98.24000 |

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">insights from this query result :</span>
<br>
<span style="font-family: 'Times New Roman', Times, serif; font-size: 20px;"> **USA** is the most country covid spread in with total cases of **103806563**   However it has highest percentage of people recovered with **98.917**%,  An important note to take here to make a decision with percentage could be misleading percentage_of_recovery in the question context as an example as more than 1.12 million people died.</span>

--------------------------------------------------------------------------------------------------------------------------------

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">In which year, month covid cases recorded globally more than others? </span>

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">year: </span>

In [ ]:
SELECT  ROW_NUMBER() OVER(ORDER BY SUM(new_cases) DESC) AS ranking, EXTRACT(YEAR FROM date) AS year,
        SUM(new_cases) AS total_cases,
        SUM(death_cases) AS total_deaths
FROM covid_analysis
WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
GROUP BY  EXTRACT(YEAR FROM date)
ORDER BY SUM(new_cases) DESC;

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Output</span>
<br>
<br>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| ranking | year | total_cases | total_deaths |
|----------|----------|----------|----------|
| 1 | 2022 | 383936529 | 1236402 |
| 2 | 2021 | 207230743 | 3583031 |
| 3 | 2020 | 83522360 | 1906031 |
| 4 | 2023 | 16550112 | 191436 |

<br>
<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">month: </span>

In [ ]:
SELECT ROW_NUMBER() OVER(ORDER BY SUM(new_cases) DESC) AS ranking, TO_CHAR(date, 'Month') AS month, EXTRACT(YEAR FROM date) AS year,
        SUM(new_cases) AS total_cases,
        SUM(death_cases) AS total_deaths
FROM covid_analysis
WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
GROUP BY  TO_CHAR(date, 'Month'), EXTRACT(YEAR FROM date)
ORDER BY SUM(new_cases) DESC
LIMIT 10;

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Output</span>
<br>
<br>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| ranking | month | year | total_cases | total_deaths |
|----------|----------|----------|----------|----------|
| 1 | January   | 2022 | 92118581 |242453|
| 2 | February  | 2022 | 59122013 |281161|
| 3 | March     | 2022 | 53268347 |182798|
| 4 | July      | 2022 | 31233554 |63610|
| 5 | April     | 2022 | 26900369 |87709|
| 6 | December  | 2021 | 26046424 |223224|
| 7 | August    | 2022 | 25942490 |77959|
| 8 | April     | 2021 | 22656009 |381889|
| 9 | August    | 2021 | 20105063 |304988|
| 10 | May      | 2021 | 19757432 |381306|

-----------------------------------------------------------------------------------------------------------------------------

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">What is the total number of infected people of covid globally? </span>

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Solution</span>

In [ ]:
SELECT EXTRACT(YEAR FROM date) AS year, SUM(new_cases) AS total_inficted_people_per_year, SUM(SUM(new_cases)) OVER (ORDER BY EXTRACT(YEAR FROM date)) AS cumulative_total
FROM covid_analysis
WHERE location NOT IN ('World', 'High income', 'Upper middle income', 'Europe', 'Asia', 'North America', 'Lower middle income', 'South America', 'European Union', 'Africa')
GROUP BY EXTRACT(YEAR FROM date)
ORDER BY year ASC;

<span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 25px;">Output</span>
<br>
<br>

<style>
table {
    font-family: "Times New Roman", Times, serif;
}
</style>

| year | total_inficted_people_per_year | cumulative_total |
|----------|----------|----------|
| 2020 | 83522360 | 83522360 |
| 2021 | 207230743 | 290753103 |
| 2022 | 383936529 | 674689632 |
| 2023 | 16550112 | 691239744 |

<br><span style="font-family: 'Times New Roman', Times, serif; font-weight: bold; font-size: 20px;">insights from this query result :</span>
<br>
<span style="font-family: 'Times New Roman', Times, serif; font-size: 20px;"> COVID-19 infected **691239744** people around the world till the date of **09-03-2023**</span>